## Libraries

In [100]:
import pandas as pd
import numpy as np

import great_expectations as gx
from great_expectations.core.batch import BatchRequest, RuntimeBatchRequest
from great_expectations.expectations.expectation import ExpectationConfiguration
from great_expectations import expectations as gxe

import re
import os

pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 50) # None

import warnings
warnings.simplefilter('ignore')

## Imports

In [102]:
import shutil
import os

# Remove previous context (necessary when re-create data context)
if os.path.exists("./context/gx"):
    shutil.rmtree("./context/gx") 

# Create GX data context
context = gx.get_context() # EphemeralDataContext
#context = gx.get_context(mode="file", project_root_dir="./context") # FileDataContext

print(type(context).__name__)

EphemeralDataContext


## Great Expectations

In [ ]:
# Dataset https://www.kaggle.com/datasets/uom190346a/sleep-health-and-lifestyle-dataset

In [103]:
source_folder = "path"
data_source_name = "my_filesystem_data_source"
data_source = context.data_sources.add_pandas(name=data_source_name)

In [104]:
print(data_source)

id: 35a03778-8693-4553-bae8-dcbf95a56c52
name: my_filesystem_data_source
type: pandas



In [105]:
asset_name = "Sleep health and lifestyle dataset"
path_to_data = "path\\Sleep_health_and_lifestyle_dataset.csv"
asset = data_source.add_csv_asset(name=asset_name, filepath_or_buffer=path_to_data)

In [106]:
dataframe = pd.read_csv(path_to_data)

In [108]:
batch_definition_name = "Sleep health and lifestyle dataset"
batch_definition = asset.add_batch_definition(
    name=batch_definition_name)

batch = batch_definition.get_batch()
batch.head()

Calculating Metrics: 100%|██████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 250.60it/s]


   Person ID Gender  Age            Occupation  Sleep Duration  \
0          1   Male   27     Software Engineer             6.1   
1          2   Male   28                Doctor             6.2   
2          3   Male   28                Doctor             6.2   
3          4   Male   28  Sales Representative             5.9   
4          5   Male   28  Sales Representative             5.9   

   Quality of Sleep  Physical Activity Level  Stress Level BMI Category  \
0                 6                       42             6   Overweight   
1                 6                       60             8       Normal   
2                 6                       60             8       Normal   
3                 4                       30             8        Obese   
4                 4                       30             8        Obese   

  Blood Pressure  Heart Rate  Daily Steps Sleep Disorder  
0         126/83          77         4200            NaN  
1         125/80          75      

### Expectations

In [109]:
# Create GX expectation suite (test suite)
suite_name = f"{data_source_name}_suite"
suite = gx.ExpectationSuite(name=suite_name)

# Add GX expectation suite to context
suite = context.suites.add(suite)

In [110]:
all_test_case = []

# ID ########################################
id_variants = ['id', 'patientid', 'patient_id', 'patient id', 'person id', 'person_id', 'personid']
id_columns = [column for column in dataframe.columns if column.lower() in id_variants]
print("Columns para Id:", id_columns)
if id_columns:
    column_name = id_columns[0]

    all_test_case.append(
        gxe.ExpectColumnValuesToBeUnique(
            column=column_name
        ))

    all_test_case.append(
        gxe.ExpectColumnValuesToNotBeNull(
            column=column_name
        ))

# Age #########################################
age_keywords = ['age', 'edad']
age_pattern = re.compile(r'|'.join(age_keywords), re.IGNORECASE)
age_columns = [column for column in dataframe.columns if age_pattern.search(column)]
print("Columns para Age:", age_columns)
for column_name in age_columns:

    all_test_case.append(
        gxe.ExpectColumnValuesToBeInTypeList(
            column=column_name,
            type_list=["int64", "float64"]
        ))

    all_test_case.append(
        gxe.ExpectColumnValueZScoresToBeLessThan(
            column=column_name,
            threshold=1.96,
            double_sided=True
        ))

    if pd.api.types.is_numeric_dtype(dataframe[column_name]):
        all_test_case.append(
            gxe.ExpectColumnValuesToBeBetween(
                column=column_name,
                min_value=0,
                max_value=120
        ))     
    
# Date ########################################
date_keywords = ['date', 'fecha', 'data']
date_pattern = re.compile(r'|'.join(date_keywords), re.IGNORECASE)
date_columns = [column for column in dataframe.columns if date_pattern.search(column)]
date_regex = ['\d{4}-(0[1-9]|1[0-2])-(0[1-9]|[12][0-9]|3[01])', # YYYY-MM-DD r"^\d{4}-\d{2}-\d{2}$"
              '(0[1-9]|1[0-2])/(0[1-9]|[12][0-9]|3[01])/\d{4}', # MM/DD/YYYY
              '(0[1-9]|[12][0-9]|3[01])/(0[1-9]|1[0-2])/\d{4}', # DD/MM/YYYY
              '(0[1-9]|[12][0-9]|3[01])-(0[1-9]|1[0-2])-\d{4}', # DD-MM-YYYY
              '(0[1-9]|[12][0-9]|3[01]).(0[1-9]|1[0-2]).\d{4}'] # DD.MM.YYYY

#format_descriptions_date = "YYYY-MM-DD, MM/DD/YYYY, DD/MM/YYYY, DD-MM-YYYY, DD.MM.YYYY"
print("Columns para Date columns:", date_columns)
for column_name in date_columns:
    all_test_case.append(
        gxe.ExpectColumnValuesToMatchRegexList(
            column=column_name,
            regex_list=date_regex,
            match_on="any"
    ))

# Sex #########################################
sex_keywords = ['sex', 'gender']
sex_pattern = re.compile(r'|'.join(sex_keywords), re.IGNORECASE)
sex_columns = [column for column in dataframe.columns if sex_pattern.search(column)]
sex_value_patterns = [
    r'^[MF]$',
    r'^(Male|Female)$',
    r'^[01]$',
    r'^[mf]$'
]
#format_descriptions_sex = "M and F, Male and Female, Mâle and Femelle, Macho and Hembra, m and f or 0 and 1"
print("Columns para Sex columns:", sex_columns)
for column_name in sex_columns:
    all_test_case.append(
        gxe.ExpectColumnValuesToMatchRegexList(
            column=column_name,
            regex_list=sex_value_patterns,
            match_on="any"
    ))
    
# BMI #########################################
bmi_pattern = re.compile(r'bmi', re.IGNORECASE)
bmi_columns = [column for column in dataframe.columns if bmi_pattern.search(column)]
print("Columns para BMI columns:", bmi_columns)
for column_name in bmi_columns:
    all_test_case.append(
        gxe.ExpectColumnValuesToBeInTypeList(
            column=column_name,
            type_list=["int64", "float64"]
    ))
    if pd.api.types.is_numeric_dtype(dataframe[column_name]):
        all_test_case.append(
            gxe.ExpectColumnValuesToBeBetween(
            column=column_name,
            min_value=10,
            max_value=60,
            mostly=0.99
        ))
    
    all_test_case.append(
        gxe.ExpectColumnValuesToBeInTypeList(
        column=column_name,
        type_list=["int64", "float64"]
    ))

# Columnas con numeros ########################
col_keywords = ['min', 'max', 'mean', 'median', 'sum', 'average', 'avg', 'media', 'height', 'weight', 'kilo', 'gram',
                'percent', 'perimeter', 'radius', 'temperatura', 'metre', 'centimetre', 'milimeter', 'ratio', 'length',
                'duration', 'rate', 'width', 'depth', 'diameter', 'distance', 'price', 'cost', 'revenue', 'income', 
                'humidity', 'pH', 'voltage', 'current']
col_pattern = re.compile(r'|'.join(col_keywords), re.IGNORECASE)
col_columns = [column for column in dataframe.columns if col_pattern.search(column)]
print("Columns para Col columns:", col_columns)
for column_name in col_columns:
    all_test_case.append(
        gxe.ExpectColumnValuesToBeInTypeList(
        column=column_name,
        type_list=["int64", "float64"]     
    ))

    all_test_case.append(
        gxe.ExpectColumnValueZScoresToBeLessThan(
        column=column_name,
        threshold=1.96, # 95%, 99% = 2.58
        double_sided=True 
    ))

for column in dataframe.columns:
    all_test_case.append(
        gxe.ExpectColumnValuesToNotBeNull(
       column=column,
       mostly=0.90
       ))


for test_case in all_test_case:
    suite.add_expectation(test_case)

Columns para Id: ['Person ID']
Columns para Age: ['Age']
Columns para Date columns: []
Columns para Sex columns: ['Gender']
Columns para BMI columns: ['BMI Category']
Columns para Col columns: ['Sleep Duration', 'Physical Activity Level', 'Heart Rate']


### Validation

In [111]:
# Create GX validation definition
definition_name = f"{data_source_name}_definition"
validation_definition = gx.ValidationDefinition(
    data=batch_definition, suite=suite, name=definition_name
)

# Add validation definition to context
validation_definition = context.validation_definitions.add(
  validation_definition
)

# Run validation (run test suite)
validation_results = validation_definition.run()

Calculating Metrics: 100%|██████████████████████████████████████████████████████████| 133/133 [00:00<00:00, 294.30it/s]


### Checkpoint

In [112]:
action_list = [
    # This Action updates the Data Docs static website with the Validation
    # Results after the Checkpoint is run.
    gx.checkpoint.UpdateDataDocsAction(
        name="update_all_data_docs",
    ),
]

# Create GX checkpoint
checkpoint_name = f"{data_source_name}_checkpoint"
checkpoint = gx.Checkpoint(
    name=checkpoint_name,
    validation_definitions=[validation_definition],
    actions=action_list,
    result_format={"result_format": "COMPLETE"},
)

# Add GX checkpint suite to context
checkpoint = context.checkpoints.add(checkpoint)

### Export report to Data_docs

In [113]:
import time

site_config = {
    "class_name": "SiteBuilder",
    "site_index_builder": {"class_name": "DefaultSiteIndexBuilder"},
    "store_backend": {
        "class_name": "TupleFilesystemStoreBackend",
        "base_directory": "path"
    },
}

# Add data docs to context
site_name = f"{data_source_name}_docs_site_{int(time.time())}"
context.add_data_docs_site(site_name=site_name, site_config=site_config)

result = checkpoint.run()

Calculating Metrics: 100%|██████████████████████████████████████████████████████████| 133/133 [00:00<00:00, 215.69it/s]
